In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

from torch.utils.data import DataLoader, Dataset 
from torchvision import datasets, transforms

# Hyperparameters
batch_size = 32
learning_rate = 0.001
epochs = 5
kernel_size = 3
stride = 2

## 모듈화  
코드의 가독성을 좋게 하고 보수 및 관리를 쉽게 합니다.  

```
+-- configs/
|   +-- cnn.yaml
+-- dataset/
|   +-- mnist_data/...
|   +-- MNIST_LOADER.py
+-- models/
|   +-- cnn.py
|   +-- res_net.py
+-- runs/
|   +-- cnn/...
|   +-- experiments/...
+-- wetights/
|   +-- cnn.pth
+-- argparse_tutorial.py
+-- README.md
+-- run_cnn.py
+-- run_cnn2.py
+-- requirements.txt
+-- utils.py
```

## Quiz (Easy)  
위 구조와 같이 모듈화를 진행하겠습니다.  
1) dataset 폴더를 만들고 mnist_data 폴더를 옮겨주세요. 그리고 MNIST_LOADER.py를 dataset에 만듭니다.   
2) models 폴더를 만들고 cnn.py, res_net.py를 만들고 내용을 채워주세요   
3) runs 폴더를 만들어주세요.   
4) weights 폴더를 만들어주세요.   
5) requirements.txt 파일을 만들어주세요.    

## Quiz (Hard)

학습에 필요한 각 모듈들을 불러와서 학습 및 테스트를 수행하세요.  

In [10]:
%%writefile run_cnn3.py

import torch
import torch.nn as nn
import torch.optim as optim
import os
import argparse
import yaml
from models.CNN import CNN
from dataset.MNIST_LOADER import make_loader

def train(epoch, model, loss_func, train_loader, optimizer):
    model.train()
    for batch_index, (x, y) in enumerate(train_loader):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        y_pred = model(x)
        loss = loss_func(y_pred, y)
        loss.backward()
        optimizer.step()
        if batch_index % 100 == 0:
            print(f'Train Epoch: {epoch+1} | Batch Status: {batch_index*len(x)}/{len(train_loader.dataset)} \
            ({100. * batch_index * batch_size / len(train_loader.dataset):.0f}% | Loss: {loss.item():.6f}')

def test(model, loss_func, test_loader):
    model.eval()
    test_loss = 0
    correct_count = 0
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        test_loss += loss_func(y_pred, y).item()
        pred = y_pred.data.max(1, keepdim=True)[1]
        # torch.eq : Computes element-wise equality. return counts value
        correct_count += pred.eq(y.data.view_as(pred)).cpu().sum()
    
    test_loss /= len(test_loader.dataset)
    print(f'=======================\n Test set: Average loss: {test_loss:.4f}, Accuracy: {correct_count/len(test_loader.dataset):.3}')

parser = argparse.ArgumentParser(description='run_cnn3')
parser.add_argument('--config_path', type=str, default='./configs/cnn.yaml', help='config file path')
args = parser.parse_args()
config_path = args.config_path

with open(config_path ,'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    print(type(config))
    
batch_size = config['batch_size']
lr = config['learning_rate']
epochs = config['epochs']
kernel_size = config['kernel_size']
stride = config['stride']

train_loader, valid_loader, test_loader, shape = make_loader(batch_size)
C = shape[0]
W = shape[1]
H = shape[2]

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
cnn = CNN(C=C, W=W, H=H, K=3, S=2) 
cnn = cnn.to(device)
ce_loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr=lr)

for epoch in range(epochs):
    train(epoch, cnn, ce_loss, train_loader, optimizer)

test(cnn, ce_loss, test_loader)

Overwriting run_cnn3.py


In [ ]:
!python run_cnn3.py --config_path ./configs/cnn.yaml